# Injectivity t/hr.bar

some intro text

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from ipywidgets import interactive, Layout, FloatSlider
from utilities import* # functions in the utilities.py file

Because the cell below includes all steps required to import and mudge the data (everything from notebook 1), it will take a little while to run. 

In [ ]:
flowrate = read_flowrate(r'Data-FlowRate.xlsx')
pts = read_pts(r'Data-PTS.xlsx')

In [ ]:
pts.info()

In [ ]:
flowrate.info()

In [ ]:
heating_37days = pd.read_csv('Data-heating37days.csv')

heating_37days['pressure_bara'] = heating_37days.pres_barg - 1

heating_37days.head(2)



In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(24,8),sharey=True)

ax1.scatter(pts.pressure_bara, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)

ax1.plot(heating_37days.pressure_bara, heating_37days.depth_m, c = 'k', label = 'Heating 37 days')

ax1.legend()

ax2.scatter(pts.datetime, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)

ax3 = ax2.twinx()
ax3.plot(flowrate.datetime, flowrate.flow_tph, 
    c='k', linestyle = '-', linewidth = 3, alpha = 0.3, 
    label='Surface pump flowrate')

ax1.set_ylim(1000,0)  # 940,400 
ax1.set_xlim(0,55)     # 20,500

ax1.set_xlabel('Pressure [bara]')
ax1.set_ylabel('Depth [m]')

ax2.set_xlabel('Time [hh:mm]')
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax3.set_ylabel('Flowrate [t/hr]')

for ax in [ax1, ax2]:
    ax.grid()

# completion test logs cross at this depth
#ax1.axhline(725,0,100, color='k',linestyle = ':');


# Pressure analysis and well pivot points

Some kind of side descrition of pivot points with the nice figure from Katie's PhD

Use the interactive plot to select the times within the completion test program when the pressure down-hole is most likely to be stable for a given flow rate

In [ ]:
#from ipywidgets import interactive, Layout, FloatSlider
# fix legend display here and in other figures

min_timestamp = pts.timestamp.iloc[0]
max_timestamp = pts.timestamp.iloc[-1]

def subselect_plot(first_value, second_value, third_value):
    f,ax1 = plt.subplots(1,1, figsize = (20,6))
    ax1.plot(pts.timestamp, pts.depth_m, c = 'k', label = 'PTS tool depth')
    ax2 = ax1.twinx()
    ax2.plot(flowrate.timestamp, flowrate.flow_tph, c='k', linestyle = ':', label='Surface pump flowrate')
    ymin = pts.depth_m.min()
    ymax = pts.depth_m.max() + 100
    ax1.vlines(first_value, ymin, ymax, color='tab:green')
    ax1.vlines(second_value, ymin, ymax, color='tab:orange')
    ax1.vlines(third_value, ymin, ymax, color='tab:red')
    ax1.set_ylim(pts.depth_m.max() + 100, 0)

result = interactive(subselect_plot,
         
         first_value = FloatSlider
         (
             value = (max_timestamp - min_timestamp)/6 + min_timestamp,
             description = '1st value',
             min = min_timestamp, 
             max = max_timestamp, 
             step = 10, 
             continuous_update=True,
             layout = Layout(width='80%'),
             ),
          
          second_value = FloatSlider
          (
             value = (max_timestamp - min_timestamp)/4 + min_timestamp, 
             description = '2nd value',
             min = min_timestamp, 
             max = max_timestamp, 
             step = 10, 
             continuous_update=True,
             layout = Layout(width='80%')
             ),

          third_value = FloatSlider
          (
             value = (max_timestamp - min_timestamp)/2 + min_timestamp, 
             description = '3rd value',
             min = min_timestamp, 
             max = max_timestamp, 
             step = 10, 
             continuous_update=True,
             layout = Layout(width='80%')
             )
)

display(result);


After you place the 1st (green), 2nd (orange), and 3rd (red) line locations, run the cell below to call the results. 

In [ ]:
# extract pressure and flow rate at the marked points
print(
    '1st value =',result.children[0].value, ' which is', datetime.fromtimestamp(result.children[0].value), 
    '\n 2nd value =', result.children[1].value, ' which is', datetime.fromtimestamp(result.children[1].value),
    '\n 3rd value =', result.children[2].value, ' which is', datetime.fromtimestamp(result.children[2].value),
    )


Because result.children will change each time you move the sliders in the plot above or re-run this Jupyter Notebook, we copy-paste our selection below. This records your choice and will be the values you do the rest of the interpretation with. 

### My selected data

1st value = 1607641784.448  which is 2020-12-11 12:09:44.448000

2nd value = 1607650624.448  which is 2020-12-11 14:37:04.448000
 
3rd value = 1607655614.448  which is 2020-12-11 16:00:14.448000

In [ ]:
# my selected data as objects

first_timestamp = 1607641784.448
second_timestamp = 1607650624.448
third_timestamp = 1607655614.448

In [ ]:
def find_index(value, df, colname):
    '''
    Find the dataframe index for the exact matching value or nearest two values

    args:   value: (float or int) the search term
            df: (obj) the name of the dataframe that is searched
            colname: (str) the name of the coloum this is searched

    returns:  dataframe index(s) for the matching value or the two adjacent values
              rows can be called from a df using df.iloc[[index_number,index_number]]
    '''
    exactmatch = df[df[colname] == value]
    if not exactmatch.empty:
        return exactmatch.index
    else:
        lowerneighbour_index = df[df[colname] < value][colname].idxmax()
        upperneighbour_index = df[df[colname] > value][colname].idxmin()
        return [lowerneighbour_index, upperneighbour_index] 

In [ ]:
# make new pts df that has values adjacent my selected value

first_pts = pts.iloc[find_index(first_timestamp, pts, 'timestamp')]
second_pts = pts.iloc[find_index(second_timestamp, pts, 'timestamp')]
third_pts = pts.iloc[find_index(third_timestamp, pts, 'timestamp')]

first_pts

In [ ]:
# make an array of the mean pressure values
import numpy as np

list = []
for df in [first_pts, second_pts, third_pts]:
    mean_pressure = df['pressure_bara'].mean()
    list.append(mean_pressure)
pressure_array = np.array(list)

print('pressure data =', pressure_array)
print('object type =', type(pressure_array))


In [ ]:
# make new flowrate df that has values adjacent my selected value

first_flowrate = flowrate.iloc[find_index(first_timestamp, flowrate, 'timestamp')]
second_flowrate = flowrate.iloc[find_index(second_timestamp, flowrate, 'timestamp')]
third_flowrate = flowrate.iloc[find_index(third_timestamp, flowrate, 'timestamp')]
first_flowrate

In [ ]:
# make an array of the mean flowrate values

list = []
for df in [first_flowrate, second_flowrate, third_flowrate]:
    mean_flowrate = df['flow_tph'].mean()
    list.append(mean_flowrate)
flowrate_array = np.array(list)

print('flowrate data =', flowrate_array)
print('object type =', type(flowrate_array))

In [ ]:
from numpy.polynomial.polynomial import polyfit

# calculate the linear fit to the pressure and flowrate arrays
polyfit_b, polyfit_m = polyfit(pressure_array, flowrate_array, 1)

# a fancy way to format strings that calls the objects polyfit_b and polyfit_m
print("The numpy polyfit linear model for our data: y = {:.5} + {:.5}X"
      .format(polyfit_b, polyfit_m))


In [ ]:
f,ax = plt.subplots(1,1,figsize=(6,4))

ax.scatter(pressure_array, flowrate_array, 
           s=100, c='k', label = 'Data')

ax.plot(pressure_array, # x values
    polyfit_m * pressure_array + polyfit_b, # use the model to generate y values
    color='tab:orange', linestyle='-', linewidth=4, alpha=0.5, label='Linear fit')

ax.text(pressure_array[0],flowrate_array[2], 
    "Injectivity index [t/hr.bar] = {:}".format(round(polyfit_m)), 
    fontsize = 12, horizontalalignment='right')

ax.legend()
ax.set_ylabel('Rate [t/hr]')
ax.set_xlabel('Pressure [bara]')  

**Include the adapted version of MAG's II plot**

***

<p><center>© 2021 <a href="https://www.cubicearth.nz/">Irene Wallis</a> and <a href="https://www.linkedin.com/in/katie-mclean-25994315/">Katie McLean</a> <a href="https://creativecommons.org/licenses/by/4.0/"</a></center></p>

<p><center>Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a></center></p>

<p><center>Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.</center></p>

***